In [1]:
import itertools
import logging
import os
import pickle

from cltk.stop.greek.stops import STOPS_LIST
import gensim
# from gensim.corpora.wikicorpus import _extract_pages, filter_wiki
from gensim.corpora.mmcorpus import MmCorpus
from gensim.utils import smart_open, simple_preprocess
import numpy as np

In [2]:
logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.INFO)
logging.root.level = logging.INFO  # ipython sometimes messes up the logging setup; restore

In [3]:
STOPS_LIST = [simple_preprocess(stop, deacc=True)[0] for stop in STOPS_LIST if len(simple_preprocess(stop, deacc=True)) > 0]
STOPS_LIST += ["τηϲ", "τοιϲ", "εϲτι", "προϲ", "ειϲ", "ταϲ", "ωϲ", "τουϲ"]

In [4]:
def tokenize(text):
    # https://radimrehurek.com/gensim/utils.html#gensim.utils.simple_preprocess
    #! todo: rm words shorter than 3
    tokens = [token for token in simple_preprocess(text, deacc=True)]
    return [token for token in tokens if token not in STOPS_LIST]
    

def iter_tlg(tlg_dir):
    file_names = os.listdir(tlg_dir)
    for file_name in file_names:
        file_path = os.path.join(tlg_dir, file_name)
        with open(file_path) as file_open:
            file_read = file_open.read()
        tokens = tokenize(file_read)
        # ignore short docs
        if len(tokens) < 50:
            continue
        yield file_name, tokens

In [5]:
tlg_preprocessed = os.path.expanduser('~/cltk_data/greek/text/tlg/plaintext/')
stream = iter_tlg(tlg_preprocessed)

In [6]:
for title, tokens in itertools.islice(iter_tlg(tlg_preprocessed), 8):
    print(title, tokens[:10])  # print the article title and its first ten tokens

TLG2346.TXT ['πολυκριτης', 'ιστορια', 'αυτη', 'εληφθη', 'αʹ', 'ανδρισκου', 'ναξιακων', 'γραφει', 'αυτης', 'θεοφραστος']
TLG1389.TXT ['αβαρις', 'ονομα', 'κυριον', 'λοιμου', 'φασι', 'πασαν', 'οικου', 'μενην', 'γεγονοτος', 'ανειλεν']
TLG0404.TXT ['ειτ', 'γυναικος', 'εστιν', 'ευνοικωτερον', 'γαμετης', 'εταιρα', 'πολυ', 'μαλ', 'εικοτως', 'νομω']
TLG0235.TXT ['εκητι', 'συλοσωντος', 'ευρυχωριη', 'πολλα', 'μεταιχμιωι', 'νοτος', 'κυλινδει', 'κυματ', 'ευρειης', 'αλος']
TLG0535.TXT ['αναιτιον', 'αιτιον', 'οιον', 'αμα', 'τουτο', 'γεγονεναι', 'τουτο', 'λαμβανουσιν', 'μαλιστα', 'ταις']
TLG0507.TXT ['σφιγγ', 'αρρεν', 'μαγειρον', 'οικιαν', 'ειληφ', 'απλως', 'μα', 'θεους', 'ων', 'λεγη']
TLG1816.TXT ['φωσφορε', 'φωσφορεουσα', 'φιλων', 'φως', 'φως', 'φερε', 'λαμπας', 'μοι', 'τεον', 'αεισαι']
TLG0476.TXT ['πυθαγοριστι', 'θυομεν', 'λοξια', 'εμψυχον', 'ουδεν', 'εσθιοντες', 'παντελως', 'βοιωτιος', 'ολιγα', 'αλλων']


# Mk word dictionaries

In [7]:
doc_stream = (tokens for _, tokens in iter_tlg(tlg_preprocessed))

In [8]:
%time id2word_tlg = gensim.corpora.Dictionary(doc_stream)
print(id2word_tlg)

INFO : adding document #0 to Dictionary(0 unique tokens: [])
INFO : built Dictionary(1161917 unique tokens: ['πολυκριτης', 'ιστορια', 'αυτη', 'εληφθη', 'αʹ']...) from 1484 documents (total 44655183 corpus positions)


CPU times: user 4min 5s, sys: 2.17 s, total: 4min 7s
Wall time: 4min 7s
Dictionary(1161917 unique tokens: ['πολυκριτης', 'ιστορια', 'αυτη', 'εληφθη', 'αʹ']...)


In [9]:
# this cutoff might lose too much info, we'll see
# ignore words that appear in less than 20 documents or more than 10% documents
id2word_tlg.filter_extremes(no_below=20, no_above=0.1)
print(id2word_tlg)

INFO : discarding 1077805 tokens: [('πολυκριτης', 4), ('ιστορια', 280), ('αυτη', 876), ('εληφθη', 150), ('αʹ', 319), ('ανδρισκου', 7), ('ναξιακων', 4), ('γραφει', 417), ('αυτης', 873), ('θεοφραστος', 150)]...
INFO : keeping 84112 tokens which were in no less than 20 and no more than 148 (=10.0%) documents
INFO : resulting dictionary: Dictionary(84112 unique tokens: ['συνεβησαν', 'ετεμνον', 'εφρουρουν', 'δηλιω', 'ερυθραιων']...)


Dictionary(84112 unique tokens: ['συνεβησαν', 'ετεμνον', 'εφρουρουν', 'δηλιω', 'ερυθραιων']...)


# Mk vectors

Now start again with the corpus, turning the actual words into integers from our map.

In [10]:
doc = "περὶ ποιητικῆς αὐτῆς τε καὶ τῶν εἰδῶν αὐτῆς, ἥν τινα δύναμιν ἕκαστον ἔχει, καὶ πῶς δεῖ συνίστασθαι τοὺς μύθους [10] εἰ μέλλει καλῶς ἕξειν ἡ ποίησις, ἔτι δὲ ἐκ πόσων καὶ ποίων ἐστὶ μορίων, ὁμοίως δὲ καὶ περὶ τῶν ἄλλων ὅσα τῆς αὐτῆς ἐστι μεθόδου, λέγωμεν ἀρξάμενοι κατὰ φύσιν πρῶτον ἀπὸ τῶν πρώτων."
doc = ' '.join(simple_preprocess(doc))
bow = id2word_tlg.doc2bow(tokenize(doc))
print(bow)

[(6873, 1), (12316, 1)]


In [11]:
print(id2word_tlg[6880], id2word_tlg[12323])

ρητορικα εργασιων


In [12]:
class TLGCorpus(object):
    def __init__(self, dump_file, dictionary, clip_docs=None):
        """Yield each document in turn, as a list of tokens (unicode strings).
        """
        self.dump_file = dump_file
        self.dictionary = dictionary
        self.clip_docs = clip_docs
    
    def __iter__(self):
        self.titles = []
        for title, tokens in itertools.islice(iter_tlg(self.dump_file), self.clip_docs):
            self.titles.append(title)
            yield self.dictionary.doc2bow(tokens)
    
    def __len__(self):
        return self.clip_docs

In [13]:
# create a stream of bag-of-words vectors
tlg_corpus = TLGCorpus(tlg_preprocessed, id2word_tlg)
vector = next(iter(tlg_corpus))
print(vector)  # print the first vector in the stream

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 2), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 2), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1)]


In [14]:
# what is the most common word in that first article?
most_index, most_count = max(vector, key=lambda _tuple: _tuple[1])
print(id2word_tlg[most_index], most_count)

μιλησιοις 2


In [15]:
# Save BoW
user_dir = os.path.expanduser('~/cltk_data/user_data/')
try:
    os.makedirs(user_dir)
except FileExistsError:
    pass
bow_path = os.path.join(user_dir, 'bow_lda_gensim.mm')

%time gensim.corpora.MmCorpus.serialize(bow_path, tlg_corpus)

INFO : storing corpus in Matrix Market format to /home/kyle/cltk_data/user_data/bow_lda_gensim.mm
INFO : saving sparse matrix to /home/kyle/cltk_data/user_data/bow_lda_gensim.mm
INFO : PROGRESS: saving document #0
INFO : PROGRESS: saving document #1000
INFO : saved 1484x84112 matrix, density=3.173% (3960894/124822208)
INFO : saving MmCorpus index to /home/kyle/cltk_data/user_data/bow_lda_gensim.mm.index


CPU times: user 3min 59s, sys: 2.37 s, total: 4min 1s
Wall time: 4min 1s


In [16]:
mm_corpus = gensim.corpora.MmCorpus(bow_path)
print(mm_corpus)

INFO : loaded corpus index from /home/kyle/cltk_data/user_data/bow_lda_gensim.mm.index
INFO : initializing corpus reader from /home/kyle/cltk_data/user_data/bow_lda_gensim.mm
INFO : accepted corpus with 1484 documents, 84112 features, 3960894 non-zero entries


MmCorpus(1484 documents, 84112 features, 3960894 non-zero entries)


In [17]:
print(next(iter(mm_corpus)))

[(0, 1.0), (1, 1.0), (2, 1.0), (3, 1.0), (4, 1.0), (5, 1.0), (6, 1.0), (7, 2.0), (8, 1.0), (9, 1.0), (10, 1.0), (11, 1.0), (12, 1.0), (13, 1.0), (14, 1.0), (15, 1.0), (16, 1.0), (17, 1.0), (18, 1.0), (19, 1.0), (20, 1.0), (21, 1.0), (22, 1.0), (23, 1.0), (24, 1.0), (25, 1.0), (26, 1.0), (27, 1.0), (28, 1.0), (29, 1.0), (30, 1.0), (31, 1.0), (32, 1.0), (33, 1.0), (34, 1.0), (35, 1.0), (36, 1.0), (37, 1.0), (38, 1.0), (39, 1.0), (40, 1.0), (41, 1.0), (42, 1.0), (43, 1.0), (44, 1.0), (45, 1.0), (46, 1.0), (47, 1.0), (48, 2.0), (49, 1.0), (50, 1.0), (51, 1.0), (52, 1.0), (53, 1.0), (54, 1.0), (55, 1.0), (56, 1.0), (57, 1.0), (58, 1.0), (59, 1.0), (60, 1.0), (61, 1.0), (62, 1.0), (63, 1.0), (64, 1.0), (65, 1.0), (66, 1.0), (67, 1.0), (68, 1.0), (69, 1.0)]


# LDA transformation

In [18]:
# Save for reuse
with open(os.path.expanduser('~/cltk_data/user_data/tlg_bow_id2word.dict'), 'rb') as file_open:
    id2word_tlg = pickle.load(file_open)

In [37]:
# Quick testing using just a part of the corpus

# use fewer documents during training, LDA is slow
# clipped_corpus = gensim.utils.ClippedCorpus(mm_corpus, 100)
%time lda_model = gensim.models.LdaMulticore(mm_corpus, num_topics=50, id2word=id2word_tlg, passes=100)

INFO : using symmetric alpha at 0.02
INFO : using symmetric eta at 1.1887779362815025e-05
INFO : using serial LDA version on this node
INFO : running online LDA training, 50 topics, 100 passes over the supplied corpus of 1484 documents, updating every 14000 documents, evaluating every ~1484 documents, iterating 50x with a convergence threshold of 0.001000
INFO : training LDA model using 7 processes
INFO : PROGRESS: pass 0, dispatched chunk #0 = documents up to #1484/1484, outstanding queue size 1
INFO : topic #48 (0.020): 0.001*"συνεργους" + 0.001*"στοχαστικως" + 0.001*"ρυθμου" + 0.000*"αλφα" + 0.000*"επισκοπος" + 0.000*"μο" + 0.000*"ϲε" + 0.000*"στον" + 0.000*"εγνωριζετο" + 0.000*"πεντεκαιδεκατον"
INFO : topic #13 (0.020): 0.001*"ροβοαμ" + 0.001*"ρυθμου" + 0.001*"αναληψιν" + 0.001*"αττικοι" + 0.001*"προστεταγμενον" + 0.001*"ευτελεις" + 0.001*"κυριας" + 0.000*"συνεργους" + 0.000*"στον" + 0.000*"εγνωριζετο"
INFO : topic #27 (0.020): 0.001*"επισκοπος" + 0.001*"ροβοαμ" + 0.001*"στοχαστικω

INFO : topic diff=2.038873, rho=0.385128
INFO : -10.564 per-word bound, 1514.3 perplexity estimate based on a held-out corpus of 1484 documents with 12049996 words
INFO : PROGRESS: pass 6, dispatched chunk #0 = documents up to #1484/1484, outstanding queue size 1
INFO : topic #17 (0.020): 0.005*"ηπειροιο" + 0.004*"σιδηριτιν" + 0.003*"τειχεος" + 0.003*"σεο" + 0.002*"λευσσων" + 0.002*"διονυσω" + 0.002*"ικοντο" + 0.002*"αστεριας" + 0.002*"αφενος" + 0.002*"φερεν"
INFO : topic #26 (0.020): 0.002*"εκτεινεν" + 0.001*"νοηματων" + 0.001*"ϲε" + 0.001*"αλμυρον" + 0.001*"καταλεξω" + 0.001*"διοικει" + 0.001*"εωντων" + 0.001*"πολους" + 0.001*"πυργων" + 0.001*"φασκουσαν"
INFO : topic #41 (0.020): 0.047*"πεντεκαιδεκατον" + 0.006*"ασματων" + 0.005*"χηρα" + 0.004*"νισου" + 0.003*"ακρωτηριον" + 0.003*"ξʹ" + 0.003*"παρερχομενοι" + 0.002*"ρʹ" + 0.002*"εκφευγουσι" + 0.002*"μυσαραν"
INFO : topic #22 (0.020): 0.002*"ρητορικη" + 0.001*"ενυλα" + 0.001*"εκαυθη" + 0.001*"ναρθηκας" + 0.001*"τελεωτερας" + 0.001*"γα

INFO : topic #15 (0.020): 0.004*"διαλογος" + 0.003*"θαυμασται" + 0.002*"μεσοτης" + 0.002*"διατελουσι" + 0.002*"εγνωκα" + 0.002*"αντιστηναι" + 0.002*"διαλογοις" + 0.002*"προδικος" + 0.002*"μεταβαλλεται" + 0.002*"γοργια"
INFO : topic #46 (0.020): 0.003*"προσηκατο" + 0.003*"αττικοι" + 0.003*"συνιστανται" + 0.002*"διαχει" + 0.002*"ποταμιοι" + 0.002*"αλοχους" + 0.001*"πανσεληνω" + 0.001*"αρχετυπου" + 0.001*"εμψυχω" + 0.001*"εκλαμποντα"
INFO : topic #16 (0.020): 0.011*"αμφισβητησιν" + 0.004*"επιρρημα" + 0.003*"συγκοπην" + 0.003*"απορουμενοις" + 0.003*"επιλογου" + 0.003*"υπερβολων" + 0.003*"ρυθμοις" + 0.002*"ευειδεις" + 0.002*"καμπανοι" + 0.002*"ονομαστι"
INFO : topic #33 (0.020): 0.003*"εξηρημενης" + 0.002*"κερως" + 0.002*"θεοπομπος" + 0.002*"αλεξις" + 0.001*"ερμιππος" + 0.001*"αντιφανης" + 0.001*"πδ" + 0.001*"συκα" + 0.001*"λωστε" + 0.001*"τετυκται"
INFO : topic diff=2.194567, rho=0.269758
INFO : -10.260 per-word bound, 1226.1 perplexity estimate based on a held-out corpus of 1484 documents

INFO : topic #19 (0.020): 0.019*"λευκοτητα" + 0.013*"μιγμα" + 0.010*"πυελος" + 0.010*"ροιας" + 0.008*"απαγγειλαι" + 0.007*"αλισκηται" + 0.006*"αδηλα" + 0.005*"κατενεχθησεται" + 0.005*"κολακευων" + 0.005*"κολλη"
INFO : topic #25 (0.020): 0.006*"συμπληροι" + 0.005*"ερμηνεια" + 0.004*"προαγορευει" + 0.003*"ανοδον" + 0.002*"ταχυν" + 0.002*"κοιμωμενους" + 0.002*"αλοχω" + 0.002*"παρεχουσα" + 0.002*"ακολαστους" + 0.002*"παρεδιδουν"
INFO : topic diff=0.933939, rho=0.225063
INFO : -10.208 per-word bound, 1183.0 perplexity estimate based on a held-out corpus of 1484 documents with 12049996 words
INFO : PROGRESS: pass 19, dispatched chunk #0 = documents up to #1484/1484, outstanding queue size 1
INFO : topic #39 (0.020): 0.008*"κωλυοντι" + 0.007*"αιτησις" + 0.005*"δηεις" + 0.005*"ανερχομενον" + 0.005*"απονον" + 0.005*"συμπληρουσθαι" + 0.004*"υβρισαντες" + 0.004*"εισαγη" + 0.004*"μουνοι" + 0.004*"αποστασεις"
INFO : topic #29 (0.020): 0.011*"υπατον" + 0.004*"εκαταιος" + 0.003*"αζ" + 0.003*"κολοφων"

INFO : topic #40 (0.020): 0.020*"τρεπων" + 0.006*"μελετωμεν" + 0.004*"χαιρω" + 0.004*"εμφερεις" + 0.003*"διερχομενου" + 0.003*"θρηνουσαν" + 0.003*"νεστοριος" + 0.003*"βιαι" + 0.003*"παρισταμενον" + 0.002*"μαθωσιν"
INFO : topic diff=0.354359, rho=0.197096
INFO : -10.191 per-word bound, 1168.8 perplexity estimate based on a held-out corpus of 1484 documents with 12049996 words
INFO : PROGRESS: pass 25, dispatched chunk #0 = documents up to #1484/1484, outstanding queue size 1
INFO : topic #0 (0.020): 0.025*"μο" + 0.012*"μοιρων" + 0.010*"ζωας" + 0.010*"ιβ" + 0.009*"ιδ" + 0.008*"στομαχου" + 0.007*"γαληνου" + 0.007*"ιγ" + 0.006*"ισαριθμων" + 0.005*"εμφαινομενα"
INFO : topic #18 (0.020): 0.032*"συνεργους" + 0.019*"αλφα" + 0.017*"δελτα" + 0.017*"εξαιτεισθαι" + 0.014*"παραδεδομενας" + 0.012*"σχισθηναι" + 0.011*"λμ" + 0.011*"αγραφα" + 0.010*"βητα" + 0.008*"λευ"
INFO : topic #3 (0.020): 0.009*"ευειδεις" + 0.007*"κελυφος" + 0.006*"κογχυλιων" + 0.006*"γενικη" + 0.006*"σπληνος" + 0.005*"διισχυριζον

...

INFO : topic diff=0.015096, rho=0.110606
INFO : -10.154 per-word bound, 1139.6 perplexity estimate based on a held-out corpus of 1484 documents with 12049996 words
INFO : PROGRESS: pass 81, dispatched chunk #0 = documents up to #1484/1484, outstanding queue size 1
INFO : topic #6 (0.020): 0.004*"διδαγματα" + 0.003*"δαμων" + 0.003*"δαμωνος" + 0.002*"ισχυε" + 0.002*"αρχαιης" + 0.002*"απολειπομενος" + 0.002*"μεμφομενη" + 0.002*"πληρουντες" + 0.002*"διαλεξεσι" + 0.002*"μαθησομεθα"
INFO : topic #12 (0.020): 0.002*"κυριας" + 0.002*"θερμοτης" + 0.001*"ρυθμου" + 0.001*"εθελησας" + 0.001*"χυμος" + 0.001*"τελευταιαι" + 0.001*"εψευσθη" + 0.001*"απελπισας" + 0.001*"συσκευην" + 0.001*"συνταττειν"
INFO : topic #3 (0.020): 0.010*"ευειδεις" + 0.007*"κελυφος" + 0.006*"γενικη" + 0.006*"κογχυλιων" + 0.006*"σπληνος" + 0.005*"διισχυριζοντο" + 0.005*"ανδρομεδας" + 0.005*"δυναστευων" + 0.004*"νικωσης" + 0.004*"επαγομενην"
INFO : topic #37 (0.020): 0.005*"προσαγωγην" + 0.003*"οριστου" + 0.003*"ακρους" + 0.003

INFO : topic #21 (0.020): 0.012*"καλλισθενης" + 0.011*"ζωιον" + 0.011*"θεραποντι" + 0.011*"τιτυον" + 0.010*"καθαρσιοις" + 0.009*"εδουλευον" + 0.008*"υδρευσασθαι" + 0.005*"ιστορουσιν" + 0.005*"μαξιμιανος" + 0.005*"εμακαριζον"
INFO : topic #2 (0.020): 0.004*"αλμυρον" + 0.004*"αρισταρχος" + 0.003*"ψιλην" + 0.002*"αιακιδην" + 0.002*"στρατιην" + 0.002*"ολεσθαι" + 0.002*"γελας" + 0.002*"σοφιης" + 0.002*"θαλιας" + 0.002*"επικαλουμαι"
INFO : topic #26 (0.020): 0.006*"εκτεινεν" + 0.004*"νοηματων" + 0.003*"εωντων" + 0.003*"καταλεξω" + 0.002*"διοικει" + 0.002*"πολους" + 0.002*"φασκουσαν" + 0.002*"πυργων" + 0.002*"ολι" + 0.002*"φιλτατοις"
INFO : topic #18 (0.020): 0.032*"συνεργους" + 0.019*"αλφα" + 0.018*"δελτα" + 0.017*"εξαιτεισθαι" + 0.014*"παραδεδομενας" + 0.012*"σχισθηναι" + 0.012*"λμ" + 0.011*"αγραφα" + 0.011*"βητα" + 0.008*"λευ"
INFO : topic #38 (0.020): 0.005*"ροβοαμ" + 0.003*"προστεταγμενον" + 0.003*"εννοω" + 0.003*"ταλαιπωροι" + 0.003*"εμηχανησατο" + 0.002*"ρυθμου" + 0.002*"ιωσηπος" + 0.0

INFO : topic #46 (0.020): 0.004*"προσηκατο" + 0.004*"συνιστανται" + 0.002*"αλοχους" + 0.002*"ποταμιοι" + 0.002*"διαχει" + 0.002*"καταλιμπανει" + 0.002*"αρχετυπου" + 0.002*"ενυλα" + 0.002*"πανσεληνω" + 0.002*"ομοιοτης"
INFO : topic #40 (0.020): 0.017*"τρεπων" + 0.006*"μελετωμεν" + 0.004*"χαιρω" + 0.003*"θρηνουσαν" + 0.003*"εμφερεις" + 0.003*"νεστοριος" + 0.003*"διερχομενου" + 0.003*"βιαι" + 0.003*"παρισταμενον" + 0.002*"μαθωσιν"
INFO : topic #4 (0.020): 0.014*"ηρακλειας" + 0.006*"εξεκαυσε" + 0.005*"φονευσαντα" + 0.005*"μηστωρ" + 0.003*"ωντινων" + 0.003*"κρατητα" + 0.002*"κρατης" + 0.002*"κυπριοι" + 0.002*"νευοντας" + 0.002*"οπισθε"
INFO : topic diff=0.011580, rho=0.102737
INFO : -10.152 per-word bound, 1137.5 perplexity estimate based on a held-out corpus of 1484 documents with 12049996 words
INFO : PROGRESS: pass 94, dispatched chunk #0 = documents up to #1484/1484, outstanding queue size 1
INFO : topic #5 (0.020): 0.003*"αναληψιν" + 0.003*"ρυθμου" + 0.003*"ροβοαμ" + 0.002*"ευτελεις" +

INFO : topic #25 (0.020): 0.005*"συμπληροι" + 0.004*"εξηρημενης" + 0.003*"ερμηνεια" + 0.003*"προαγορευει" + 0.002*"αγνοησαι" + 0.002*"ανοδον" + 0.002*"πετεσθαι" + 0.002*"αιγες" + 0.002*"κομψοι" + 0.002*"φασκοντος"
INFO : topic diff=0.010413, rho=0.099631
INFO : -10.151 per-word bound, 1136.7 perplexity estimate based on a held-out corpus of 1484 documents with 12049996 words


CPU times: user 5h 36min 30s, sys: 6h 57min 23s, total: 12h 33min 53s
Wall time: 4h 9min 35s


In [38]:
lda_model.print_topics(-1)  # print a few most important words for each LDA topic

INFO : topic #0 (0.020): 0.024*"μο" + 0.014*"μοιρων" + 0.011*"ιβ" + 0.011*"ζωας" + 0.009*"ιδ" + 0.008*"γαληνου" + 0.008*"στομαχου" + 0.007*"ιγ" + 0.007*"ισαριθμων" + 0.006*"εμφαινομενα"
INFO : topic #1 (0.020): 0.002*"επιτασεις" + 0.002*"φευγοντος" + 0.002*"δημοσθενους" + 0.002*"ψηφισμα" + 0.002*"αισχινης" + 0.002*"αντιτιθεις" + 0.002*"σαφει" + 0.002*"αδικημα" + 0.002*"συντελουντα" + 0.002*"κατεχοντες"
INFO : topic #2 (0.020): 0.004*"αλμυρον" + 0.004*"αρισταρχος" + 0.003*"ψιλην" + 0.002*"αιακιδην" + 0.002*"στρατιην" + 0.002*"ολεσθαι" + 0.002*"γελας" + 0.002*"σοφιης" + 0.002*"θαλιας" + 0.002*"επικαλουμαι"
INFO : topic #3 (0.020): 0.010*"ευειδεις" + 0.007*"κελυφος" + 0.006*"γενικη" + 0.006*"κογχυλιων" + 0.006*"σπληνος" + 0.005*"διισχυριζοντο" + 0.005*"ανδρομεδας" + 0.005*"δυναστευων" + 0.004*"νικωσης" + 0.004*"επαγομενην"
INFO : topic #4 (0.020): 0.014*"ηρακλειας" + 0.006*"εξεκαυσε" + 0.006*"φονευσαντα" + 0.005*"μηστωρ" + 0.003*"ωντινων" + 0.003*"κρατητα" + 0.002*"κρατης" + 0.002*"κυπριο

INFO : topic #39 (0.020): 0.025*"ετητυμον" + 0.011*"κωλυοντι" + 0.009*"αιτησις" + 0.006*"πεζων" + 0.006*"υβρισαντες" + 0.006*"συμπληρουσθαι" + 0.005*"ιππεας" + 0.005*"εισαγη" + 0.004*"ανερχομενον" + 0.004*"δηεις"
INFO : topic #40 (0.020): 0.017*"τρεπων" + 0.006*"μελετωμεν" + 0.004*"χαιρω" + 0.003*"θρηνουσαν" + 0.003*"εμφερεις" + 0.003*"νεστοριος" + 0.003*"διερχομενου" + 0.003*"βιαι" + 0.003*"παρισταμενον" + 0.002*"μαθωσιν"
INFO : topic #41 (0.020): 0.135*"πεντεκαιδεκατον" + 0.013*"ακριβεστερος" + 0.012*"ακρωτηριον" + 0.010*"εαρινης" + 0.010*"ξʹ" + 0.010*"παρερχομενοι" + 0.010*"σκαμανδρον" + 0.010*"ασματων" + 0.010*"ρʹ" + 0.009*"χηρα"
INFO : topic #42 (0.020): 0.004*"ιλλυρικον" + 0.002*"κατεχουσιν" + 0.002*"γαιος" + 0.002*"κοσμεισθαι" + 0.002*"συναρμοσας" + 0.002*"ληιζομενος" + 0.002*"ημφιεστο" + 0.002*"αφορων" + 0.002*"αποδωσω" + 0.002*"πειν"
INFO : topic #43 (0.020): 0.187*"στοχαστικως" + 0.072*"νεται" + 0.057*"αφιεσθαι" + 0.046*"πολιες" + 0.039*"επιμενη" + 0.017*"καταπετασματος" + 0.

[(0,
  '0.024*"μο" + 0.014*"μοιρων" + 0.011*"ιβ" + 0.011*"ζωας" + 0.009*"ιδ" + 0.008*"γαληνου" + 0.008*"στομαχου" + 0.007*"ιγ" + 0.007*"ισαριθμων" + 0.006*"εμφαινομενα"'),
 (1,
  '0.002*"επιτασεις" + 0.002*"φευγοντος" + 0.002*"δημοσθενους" + 0.002*"ψηφισμα" + 0.002*"αισχινης" + 0.002*"αντιτιθεις" + 0.002*"σαφει" + 0.002*"αδικημα" + 0.002*"συντελουντα" + 0.002*"κατεχοντες"'),
 (2,
  '0.004*"αλμυρον" + 0.004*"αρισταρχος" + 0.003*"ψιλην" + 0.002*"αιακιδην" + 0.002*"στρατιην" + 0.002*"ολεσθαι" + 0.002*"γελας" + 0.002*"σοφιης" + 0.002*"θαλιας" + 0.002*"επικαλουμαι"'),
 (3,
  '0.010*"ευειδεις" + 0.007*"κελυφος" + 0.006*"γενικη" + 0.006*"κογχυλιων" + 0.006*"σπληνος" + 0.005*"διισχυριζοντο" + 0.005*"ανδρομεδας" + 0.005*"δυναστευων" + 0.004*"νικωσης" + 0.004*"επαγομενην"'),
 (4,
  '0.014*"ηρακλειας" + 0.006*"εξεκαυσε" + 0.006*"φονευσαντα" + 0.005*"μηστωρ" + 0.003*"ωντινων" + 0.003*"κρατητα" + 0.002*"κρατης" + 0.002*"κυπριοι" + 0.002*"νευοντας" + 0.002*"οπισθε"'),
 (5,
  '0.003*"αναληψιν" + 0.00

In [39]:
# cache the transformed corpora to disk, for use in later notebooks
path_lda = os.path.join(user_dir, 'gensim_tlg_lda.mm')
%time gensim.corpora.MmCorpus.serialize(path_lda, lda_model[mm_corpus])

INFO : storing corpus in Matrix Market format to /home/kyle/cltk_data/user_data/gensim_tlg_lda.mm
INFO : saving sparse matrix to /home/kyle/cltk_data/user_data/gensim_tlg_lda.mm
INFO : PROGRESS: saving document #0
INFO : PROGRESS: saving document #1000
INFO : saved 1484x50 matrix, density=10.020% (7435/74200)
INFO : saving MmCorpus index to /home/kyle/cltk_data/user_data/gensim_tlg_lda.mm.index


CPU times: user 47.4 s, sys: 1min 32s, total: 2min 19s
Wall time: 18.4 s


In [40]:
# transform text into the bag-of-words space
bow_vector = id2word_tlg.doc2bow(tokenize(doc))
print([(id2word_tlg[id], count) for id, count in bow_vector])

[('ποιητικης', 1), ('μεθοδου', 1)]


In [41]:
# transform into LDA space
lda_vector = lda_model[bow_vector]
print(lda_vector)

[(22, 0.67333333333333334)]


In [42]:
# print the document's single most prominent LDA topic
print(lda_model.print_topic(max(lda_vector, key=lambda item: item[1])[0]))

0.002*"εκαυθη" + 0.002*"γαυριων" + 0.002*"ρητορικη" + 0.001*"επισκεψις" + 0.001*"συντελειται" + 0.001*"ναρθηκας" + 0.001*"κλεανθης" + 0.001*"συμβουλευτικον" + 0.001*"εδωκαμεν" + 0.001*"συνθηκη"


In [43]:
path_lda = os.path.join(user_dir, 'gensim_tlg_lda.model')

# store all trained models to disk
lda_model.save(path_lda)

INFO : saving LdaState object under /home/kyle/cltk_data/user_data/gensim_tlg_lda.model.state, separately None
INFO : saved /home/kyle/cltk_data/user_data/gensim_tlg_lda.model.state
INFO : saving LdaMulticore object under /home/kyle/cltk_data/user_data/gensim_tlg_lda.model, separately ['expElogbeta', 'sstats']
INFO : storing np array 'expElogbeta' to /home/kyle/cltk_data/user_data/gensim_tlg_lda.model.expElogbeta.npy
INFO : not storing attribute id2word
INFO : not storing attribute state
INFO : not storing attribute dispatcher
INFO : saved /home/kyle/cltk_data/user_data/gensim_tlg_lda.model


# Evaluation

# Word intrusion

> For each trained topic, they take its first ten words, then substitute one of them with another, randomly chosen word (intruder!) and see whether a human can reliably tell which one it was. If so, the trained topic is topically coherent (good); if not, the topic has no discernible theme (bad)

In [44]:
# select top 50 words for each of the 20 LDA topics
top_words = [[word for word, _ in lda_model.show_topic(topicno, topn=50)] for topicno in range(lda_model.num_topics)]
print(top_words)

[['μο', 'μοιρων', 'ιβ', 'ζωας', 'ιδ', 'γαληνου', 'στομαχου', 'ιγ', 'ισαριθμων', 'εμφαινομενα', 'συρακουσιους', 'τραπεζουντα', 'αγραφα', 'ειδαρ', 'προεχοντων', 'θινα', 'απτομεθα', 'ποιμαινοντες', 'εσθησι', 'ελαχιστω', 'σισυφος', 'νο', 'κγ', 'δυμη', 'σικυωνιας', 'καθευδει', 'αρχινος', 'πασασθαι', 'αφηρησθω', 'αρχι', 'καλυμμα', 'πολιτευομενην', 'αλεξανδρειας', 'υπεροχη', 'υποθεμενος', 'ελκομενους', 'πολον', 'στιβαδας', 'ευσεβη', 'αχθησονται', 'υπακουοντα', 'εσβεννυτο', 'μεροης', 'αντολαι', 'συντελουμενων', 'οικησαντας', 'υποτιθεσθαι', 'αξ', 'λϛʹ', 'παρηχθη'], ['επιτασεις', 'φευγοντος', 'δημοσθενους', 'ψηφισμα', 'αισχινης', 'αντιτιθεις', 'σαφει', 'αδικημα', 'συντελουντα', 'κατεχοντες', 'τελεωτερας', 'αντιθεσις', 'ρητορικη', 'θηβαιων', 'υπηρξε', 'αισχινου', 'προβολη', 'ρητορος', 'γερως', 'φιλιππω', 'οποιονουν', 'θηβαιους', 'μειδιου', 'συμφωνη', 'δημοσθενει', 'συμβουλευων', 'ναρθηκας', 'οριστου', 'δημοσθενην', 'κρινομενον', 'λυσεων', 'ασεβη', 'μοιχον', 'ψευδων', 'εκβασεως', 'τριηρεις', 'ισοκ

In [45]:
# get all top 50 words in all 20 topics, as one large set
all_words = set(itertools.chain.from_iterable(top_words))

print("Can you spot the misplaced word in each topic?")

# for each topic, replace a word at a different index, to make it more interesting
replace_index = np.random.randint(0, 10, lda_model.num_topics)

replacements = []
for topicno, words in enumerate(top_words):
    other_words = all_words.difference(words)
    replacement = np.random.choice(list(other_words))
    replacements.append((words[replace_index[topicno]], replacement))
    words[replace_index[topicno]] = replacement
    print("%i: %s" % (topicno, ' '.join(words[:10])))

Can you spot the misplaced word in each topic?
0: μο μοιρων ιβ ζωας ιδ γαληνου στομαχου ιγ ισαριθμων κομψως
1: επιτασεις φευγοντος γαυριων ψηφισμα αισχινης αντιτιθεις σαφει αδικημα συντελουντα κατεχοντες
2: αλμυρον σπευδε ψιλην αιακιδην στρατιην ολεσθαι γελας σοφιης θαλιας επικαλουμαι
3: ευειδεις κελυφος γενικη κογχυλιων κακο διισχυριζοντο ανδρομεδας δυναστευων νικωσης επαγομενην
4: ηρακλειας εξεκαυσε φονευσαντα ανωνυμοι ωντινων κρατητα κρατης κυπριοι νευοντας οπισθε
5: αναληψιν ρυθμου ροβοαμ ευτελεις απατηθεις προστεταγμενον στον εννοω ερωτωμενοι γινωσκονται
6: διδαγματα δαμων δαμωνος ισχυε απολειπομενος αρχαιης μεμφομενη βητα διαλεξεσι μαθησομεθα
7: ημαρ ταπητες γενικη νεφελαις χρονικον κατεστρεψεν φυλαξι εγνωριζετο θουκυδιδη υπεριδης
8: λεπτου μακροτερος μο ινδικης ωνομαζε ελαια ατλαντιδος προσθεντος κτησιας τειχισαι
9: καμνεις αυτιχ αισθητικη βδελυττεται ακτιον νοσερον κατεπληξε ειρξαι δραχμας τιθεμενης
10: πηδαν ιλλυρικον κολπων πελοποννησον κοσμεισθαι οχυρωματι τομον κατεχουσιν ο

In [46]:
print("Actual replacements were:")
print(list(enumerate(replacements)))

Actual replacements were:
[(0, ('εμφαινομενα', 'κομψως')), (1, ('δημοσθενους', 'γαυριων')), (2, ('αρισταρχος', 'σπευδε')), (3, ('σπληνος', 'κακο')), (4, ('μηστωρ', 'ανωνυμοι')), (5, ('εγνωριζετο', 'απατηθεις')), (6, ('πληρουντες', 'βητα')), (7, ('παιονιας', 'ημαρ')), (8, ('φιλιου', 'μο')), (9, ('βραχυτατης', 'καμνεις')), (10, ('επανερχομενος', 'πελοποννησον')), (11, ('ζοφω', 'εσθλων')), (12, ('εθελησας', 'κοινωνησειν')), (13, ('πεποιθοτες', 'ορφανους')), (14, ('επεσχον', 'λωστε')), (15, ('θαυμασται', 'κινουνται')), (16, ('ρυθμοις', 'μοιραις')), (17, ('ικοντο', 'ζωιον')), (18, ('βητα', 'λαμπραι')), (19, ('λευκοτητα', 'συμβεβηκοτος')), (20, ('υποκειμενος', 'ανεχονται')), (21, ('εμακαριζον', 'ηρωικα')), (22, ('εδωκαμεν', 'ξεινων')), (23, ('ευκαιριαν', 'πδ')), (24, ('σχοινων', 'γεννωμενος')), (25, ('εξηρημενης', 'κειμενω')), (26, ('πυργων', 'ροιας')), (27, ('καταβαινον', 'αυλως')), (28, ('υπακουοντα', 'δικαι')), (29, ('εκαταιος', 'συντιθεναι')), (30, ('οριστεον', 'ερπει')), (31, ('ερηται',

## Split doc

> We'll split each document into two parts, and check that 1) topics of the first half are similar to topics of the second 2) halves of different documents are mostly dissimilar

In [47]:
# evaluate on 1k documents **not** used in LDA training
tlg_preprocessed = os.path.expanduser('~/cltk_data/greek/text/tlg/plaintext/')
doc_stream = (tokens for _, tokens in iter_tlg(tlg_preprocessed))  # generator
test_docs = list(itertools.islice(doc_stream, 100, 200))  # ['πανυ', 'καλως', ...], [...], ...]

In [48]:
def intra_inter(model, test_docs, num_pairs=10000):
    # split each test document into two halves and compute topics for each half
    part1 = [model[id2word_tlg.doc2bow(tokens[: len(tokens) // 2])] for tokens in test_docs]
    part2 = [model[id2word_tlg.doc2bow(tokens[len(tokens) // 2 :])] for tokens in test_docs]
    
    # print computed similarities (uses cossim)
    print("average cosine similarity between corresponding parts (higher is better):")
    print(np.mean([gensim.matutils.cossim(p1, p2) for p1, p2 in zip(part1, part2)]))

    random_pairs = np.random.randint(0, len(test_docs), size=(num_pairs, 2))
    print("average cosine similarity between {} random parts (lower is better):".format(num_pairs))    
    print(np.mean([gensim.matutils.cossim(part1[i[0]], part2[i[1]]) for i in random_pairs]))

In [49]:
print("LDA results:")
intra_inter(lda_model, test_docs)

LDA results:
average cosine similarity between corresponding parts (higher is better):
0.750338842199
average cosine similarity between 10000 random parts (lower is better):
0.433126791694


# Transform all docs

In [50]:
for title, tokens in iter_tlg(tlg_preprocessed):
    #print(title, tokens[:10])  # print the article title and its first ten tokens
    print(title)
    print(lda_model[id2word_tlg.doc2bow(tokens)])
    print('')

TLG2346.TXT
[(14, 0.39389740707735388), (24, 0.21067455806178731), (29, 0.1136918037101812), (33, 0.17409762749558913), (40, 0.095309836531801279)]

TLG1389.TXT
[(1, 0.26942331363740823), (8, 0.015139339272878569), (14, 0.01973173879480002), (16, 0.089583904671040562), (24, 0.038989333443285451), (29, 0.12612947608578889), (33, 0.17598846769899817), (43, 0.2649070781529877)]

TLG0404.TXT
[(24, 0.13841483006409908), (31, 0.069124241336902051), (33, 0.76205471720457707), (44, 0.019940601431003298)]

TLG0235.TXT
[(8, 0.037968259054561616), (16, 0.18147176836703305), (21, 0.01545021771768587), (43, 0.061998877321973234), (44, 0.39413830205480471), (48, 0.2960313990133534)]

TLG0535.TXT
[(1, 0.45536139471199766), (2, 0.018797458921507026), (14, 0.095276860187092), (24, 0.30209088967983838), (33, 0.080068597297550878), (47, 0.020818225064305491)]

TLG0507.TXT
[(39, 0.97930838894220751), (44, 0.011719648440970329)]

TLG1816.TXT
[(10, 0.020210517185290437), (16, 0.031779850265826096), (17, 0.4

TLG3070.TXT
[(2, 0.011238929416186804), (5, 0.058678063663210392), (8, 0.01874374857278896), (9, 0.019893755165342857), (14, 0.105431140055585), (22, 0.019875942495514894), (24, 0.14942021334885897), (27, 0.010278129048642766), (38, 0.074411937666920197), (44, 0.018549755497763405), (46, 0.012100053827103042), (47, 0.43223233753932988)]

TLG1426.TXT
[(1, 0.18771139346164889), (5, 0.044499867349516949), (8, 0.037794828084757079), (11, 0.010359989551775509), (12, 0.049897588077488751), (14, 0.17377642530681947), (22, 0.022368234945693524), (24, 0.12943069742963384), (31, 0.18554864412178637), (40, 0.063030594068426454), (44, 0.027077761350131468), (46, 0.065401935435994291)]

TLG1641.TXT
[(5, 0.35524257201396847), (8, 0.010423069027581014), (9, 0.043430657873092235), (14, 0.11387843939602446), (22, 0.10858343140200308), (24, 0.15825087792073758), (27, 0.056081666831608472), (37, 0.063301117724056347), (38, 0.060655940116705033), (46, 0.022758200262082279)]

TLG5040.TXT
[(1, 0.02997200137

TLG3045.TXT
[(5, 0.10034237870964828), (8, 0.035410094239831415), (12, 0.010694490484758562), (14, 0.14473834226663421), (22, 0.035516257095493636), (23, 0.019430566578952813), (24, 0.11966486884312534), (27, 0.016571943393430461), (29, 0.042867618464515825), (33, 0.020595280449640395), (37, 0.020703163827741611), (38, 0.2127538118394203), (44, 0.02086517182206923), (46, 0.032050672089183614), (47, 0.09124409549738742)]

TLG0632.TXT
[(1, 0.014130143426407343), (5, 0.052620346119288328), (8, 0.035960882952038392), (9, 0.019073464097406737), (12, 0.014068559658593828), (14, 0.086916130558796725), (16, 0.019983492291534632), (17, 0.021343116696380515), (20, 0.014391127433832671), (22, 0.079257571324369766), (24, 0.2154066757104463), (28, 0.015905927298272986), (29, 0.021980928040708798), (31, 0.010594551371093981), (33, 0.12379397013237944), (35, 0.014673254034304794), (37, 0.016620379692754182), (38, 0.021679046911669688), (44, 0.066435824211923081), (45, 0.013819562217004434), (46, 0.03

[(0, 0.010128282340030008), (1, 0.075496276292893821), (5, 0.045225741039651719), (8, 0.048241840951194694), (14, 0.1143001925041815), (22, 0.029246582204396989), (24, 0.18932328687805128), (29, 0.11447521920031263), (33, 0.2731133428833924), (46, 0.011251641638823423), (47, 0.023903369958303462)]

TLG4236.TXT
[(1, 0.17800270347342562), (14, 0.011519585145488556), (16, 0.012813753437479179), (22, 0.32401834389518108), (24, 0.14273783080857241), (28, 0.018785553399724497), (30, 0.085038758075142482), (31, 0.036739575687546017), (33, 0.051418960980841463), (37, 0.033303070340390838), (38, 0.039526633141425051), (46, 0.026920723764002043), (47, 0.017494492097684392)]

TLG1923.TXT
[(1, 0.11919680218193027), (5, 0.098840579819661045), (8, 0.08222548547078172), (9, 0.01914502784607796), (14, 0.071157600377433003), (16, 0.02193064190711648), (20, 0.011819622293464168), (22, 0.033492424824580883), (24, 0.30455936989495136), (33, 0.11253552206862423), (38, 0.042171742679576629), (42, 0.03433284

TLG4080.TXT
[(2, 0.023493113203618844), (5, 0.08066961314457767), (8, 0.032384051920073854), (9, 0.16213954353008678), (12, 0.015987874695790869), (14, 0.043368139598694094), (16, 0.043929528000721904), (17, 0.010880707731188384), (19, 0.023160678559875986), (22, 0.049891138082270053), (24, 0.21938324413718488), (25, 0.011541326966846004), (28, 0.02856433672422877), (29, 0.011998806945893999), (33, 0.065756936254298931), (37, 0.021822852602863946), (38, 0.015465747182400867), (44, 0.030251821300416468), (46, 0.022684103503956363), (47, 0.041080514908380175)]

TLG0064.TXT
[(8, 0.23968660306601924), (9, 0.058499182831260099), (14, 0.043828426203045841), (19, 0.02339145724137584), (22, 0.16062568455418275), (24, 0.19264328073413708), (29, 0.018983959687474516), (31, 0.045991052511224602), (35, 0.027354543290871253), (40, 0.054264901022988407), (47, 0.12891001333503255)]

TLG1320.TXT
[(1, 0.04446603607485268), (2, 0.012718494232913685), (5, 0.07942768536734221), (8, 0.019432688275271278), 

...

TLG4193.TXT
[(1, 0.044075509346151877), (2, 0.044476163080499953), (5, 0.092637659936651698), (9, 0.04727569317484627), (12, 0.023481165110077105), (14, 0.042436880911782979), (16, 0.015112323189897338), (17, 0.010544665447395405), (22, 0.16048397873559908), (24, 0.14045684121216417), (28, 0.021968855516743678), (29, 0.013401200356360672), (30, 0.027525889942964663), (31, 0.051367004917798074), (33, 0.023875530596037947), (37, 0.1021359279992113), (38, 0.01484916311982961), (42, 0.013922082389668999), (44, 0.019251214677830517), (46, 0.033915788934908951), (47, 0.020565052430290179)]

TLG0007.TXT
[(1, 0.030804554203538344), (2, 0.015902596999201921), (5, 0.075781326805539978), (8, 0.022213046308382247), (9, 0.026855036548383327), (12, 0.017376016843769555), (14, 0.17470320577356141), (16, 0.018672216243520421), (22, 0.049517405112500266), (24, 0.31180419329242115), (29, 0.025787890173297821), (33, 0.050764982161751147), (37, 0.016746308558203474), (38, 0.021075979842894006), (42, 0.010

TLG2061.TXT
[(5, 0.24407577288979931), (8, 0.016990620399599185), (9, 0.013752641192622483), (12, 0.062013645177982725), (14, 0.085056406710476345), (16, 0.026408839018320994), (17, 0.016434246917973711), (22, 0.060630249073061401), (24, 0.18578000728354882), (27, 0.011711639492129824), (31, 0.012957314188995941), (33, 0.022530134332144007), (37, 0.017262060932209777), (38, 0.015527861978839581), (42, 0.021580059628667381), (44, 0.044097570614626463), (46, 0.021758859225098525), (47, 0.068294501715959521), (48, 0.012045717044014064)]

TLG0340.TXT
[(2, 0.18941952032050072), (8, 0.14453513971996682), (12, 0.15541637183397225), (16, 0.26790690912479903), (29, 0.19716561325289314), (34, 0.022992343183764155)]

TLG0384.TXT
[(5, 0.12216144641422845), (10, 0.011962462520151516), (12, 0.043908268856233862), (19, 0.012713094367238626), (22, 0.092897781749529743), (24, 0.18396614919518126), (29, 0.028753437150236417), (33, 0.03219631121017992), (37, 0.010739593566794697), (44, 0.0386053314581702

TLG0614.TXT
[(2, 0.025826887308763532), (5, 0.086332452621855155), (9, 0.025107048307930903), (12, 0.039219912304550221), (14, 0.071825144862140991), (16, 0.041296378057337491), (17, 0.045513280801422029), (22, 0.041480029955482396), (24, 0.28970838592930009), (25, 0.034541540678807457), (29, 0.038892638626201778), (33, 0.055424661948557753), (37, 0.013923537288561584), (44, 0.075812184297162494), (46, 0.011324088644207539), (47, 0.025775467636507822), (48, 0.041653035568258015)]

TLG0014.TXT
[(1, 0.27032481229946043), (2, 0.013789486729349166), (5, 0.071476277001895877), (8, 0.014596947498172447), (9, 0.016954739527207708), (12, 0.016601296148103278), (14, 0.11592318713909743), (16, 0.012925198192457804), (22, 0.031799566390153848), (24, 0.22997964893317116), (31, 0.021865230076376151), (33, 0.033178878008457396), (38, 0.025185105877304294), (44, 0.036400908834527249), (46, 0.013337627131579356), (47, 0.018117657205959255)]

TLG3181.TXT
[(5, 0.1275832031877713), (8, 0.0173831767772616

TLG4031.TXT
[(1, 0.041754658538487144), (2, 0.023929113630109196), (3, 0.010868247045780887), (5, 0.12467362698659688), (8, 0.017337603746144965), (9, 0.044690851734846194), (14, 0.029059333396407625), (16, 0.015782856254760565), (22, 0.18502352134212224), (24, 0.15985888879295498), (28, 0.018649236036107606), (30, 0.016739336108005209), (31, 0.026310330396136564), (37, 0.10796301831293934), (38, 0.020606738164776366), (42, 0.013466169838302303), (44, 0.016546685798059693), (46, 0.052515322019799027), (47, 0.019706204090753354)]

TLG2236.TXT
[(2, 0.032908598009874647), (5, 0.074831670298675962), (9, 0.014034812923507207), (12, 0.012003986674798873), (14, 0.26059497351944044), (22, 0.027005541829956178), (24, 0.26526603563917273), (29, 0.01797058296923695), (38, 0.07284013831441162), (40, 0.021576342686329665), (42, 0.02415906485229586), (44, 0.012946241669026595), (46, 0.013054972475719084), (47, 0.10454186832555447)]

TLG0645.TXT
[(1, 0.010115682501988006), (2, 0.024983124571434812), 

TLG2023.TXT
[(1, 0.015504952217742082), (3, 0.015981752933624197), (5, 0.11843611747068887), (8, 0.017160786877246766), (9, 0.037467939579104319), (12, 0.019502094443590082), (14, 0.061157746007637674), (16, 0.019761014669459516), (20, 0.011977824755023014), (22, 0.11523181686731272), (24, 0.18427902673266885), (28, 0.011725011230469233), (29, 0.012290285122297247), (31, 0.029389963379409297), (33, 0.048222335195405107), (37, 0.06661502490147414), (38, 0.013310188673714224), (42, 0.018259513952955327), (44, 0.01502969119038031), (46, 0.082757999937136117), (47, 0.017211045120405825), (48, 0.019644260511213898)]

TLG2506.TXT
[(0, 0.14557247151516051), (3, 0.05699168014170955), (19, 0.017857678828058908), (27, 0.15255301908480695), (31, 0.21869492872773549), (35, 0.19678571428571415), (37, 0.17424579679511171), (39, 0.022298710621702012)]

TLG1127.TXT
[(0, 0.013108139605964048), (1, 0.017576546213180281), (2, 0.045810106528874378), (3, 0.014698332337778737), (5, 0.12758389027179751), (9,

# Visualization

Following: http://nbviewer.jupyter.org/github/bmabey/pyLDAvis/blob/master/notebooks/pyLDAvis_overview.ipynb

In [51]:
lda_model.show_topics()

[(16,
  '0.009*"αμφισβητησιν" + 0.004*"αττικοι" + 0.003*"επιρρημα" + 0.003*"συγκοπην" + 0.002*"επιλογου" + 0.002*"υπερβολων" + 0.002*"απορουμενοις" + 0.002*"ρυθμοις" + 0.002*"δακνουσι" + 0.002*"φατνας"'),
 (44,
  '0.003*"ηλθ" + 0.002*"αισχυλος" + 0.002*"βροτοις" + 0.002*"αζυμα" + 0.001*"ακινητους" + 0.001*"κολαζεσθαι" + 0.001*"μνησθησομαι" + 0.001*"οδ" + 0.001*"φαλαγγες" + 0.001*"αλιας"'),
 (40,
  '0.017*"τρεπων" + 0.006*"μελετωμεν" + 0.004*"χαιρω" + 0.003*"θρηνουσαν" + 0.003*"εμφερεις" + 0.003*"νεστοριος" + 0.003*"διερχομενου" + 0.003*"βιαι" + 0.003*"παρισταμενον" + 0.002*"μαθωσιν"'),
 (3,
  '0.010*"ευειδεις" + 0.007*"κελυφος" + 0.006*"γενικη" + 0.006*"κογχυλιων" + 0.006*"σπληνος" + 0.005*"διισχυριζοντο" + 0.005*"ανδρομεδας" + 0.005*"δυναστευων" + 0.004*"νικωσης" + 0.004*"επαγομενην"'),
 (15,
  '0.015*"γοργια" + 0.009*"δηλουσαι" + 0.009*"φαιδρω" + 0.007*"μεταβαλλεται" + 0.007*"εισκρινεσθαι" + 0.007*"διαλογος" + 0.005*"διαλογοις" + 0.005*"θαυμασται" + 0.004*"διατελουσι" + 0.004*"μεσοτη

In [52]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

In [53]:
pyLDAvis.gensim.prepare(lda_model, mm_corpus, id2word_tlg)

/home/kyle/cltk/venv_pypi/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
5      13.542856        1       1 -0.169532 -0.095160
24      8.412300        1       2 -0.164909  0.030365
12      7.678061        1       3 -0.191272 -0.072763
9       6.705500        1       4  0.043452 -0.050582
14      6.005562        1       5 -0.156381 -0.007147
37      5.881227        1       6  0.075833 -0.187014
2       4.061223        1       7  0.046892  0.200834
13      3.142303        1       8 -0.161428 -0.106798
46      3.021159        1       9  0.020528 -0.173611
47      3.005663        1      10 -0.144549 -0.006327
38      2.808176        1      11 -0.129843 -0.107699
16      2.613073        1      12  0.053787  0.125828
1       2.594650        1      13 -0.115223 -0.073917
3       2.500714        1      14  0.114782  0.111345
33      2.368397        1      15 -0.024502  0.099060
22      2.330587        1      16  0.007636 -0.109740
31      2.248015        1      17 -0.071512 -0.052507
44      2.218454        1      18 -0.021421  0.147126
29      2.112818        1      19  0.012832  0.134266
42      2.063056        1      20 -0.132434 -0.042301
27      1.954767        1      21 -0.126303 -0.131151
18      1.770699        1      22  0.311466 -0.171895
48      1.583686        1      23  0.172574  0.304619
8       1.256599        1      24  0.033228  0.060617
17      1.131964        1      25  0.143122  0.245975
0       0.988146        1      26  0.287415 -0.200060
40      0.881462        1      27 -0.071871  0.028906
30      0.760484        1      28  0.129142 -0.157723
19      0.633219        1      29  0.147696  0.025164
28      0.623703        1      30  0.124170 -0.120788
11      0.539050        1      31  0.034755  0.149833
20      0.531794        1      32  0.250455 -0.139543
25      0.472317        1      33 -0.024376  0.035623
10      0.321384        1      34 -0.051404  0.031594
43      0.229487        1      35  0.140912 -0.009162
45      0.195667        1      36  0.016835 -0.013201
36      0.147815        1      37  0.029204 -0.000454
35      0.141411        1      38  0.040716  0.020428
39      0.114594        1      39  0.080628  0.040206
7       0.096786        1      40 -0.002196  0.018024
23      0.069789        1      41  0.002739  0.033094
41      0.062522        1      42  0.013624  0.018899
34      0.053785        1      43  0.006070  0.016744
21      0.044049        1      44 -0.062246  0.004080
32      0.017296        1      45 -0.070273  0.049306
26      0.016948        1      46 -0.084274  0.016928
49      0.016391        1      47 -0.081870  0.028486
15      0.015348        1      48 -0.080570 -0.004257
6       0.007632        1      49 -0.102638  0.018963
4       0.007414        1      50 -0.099468  0.037486, topic_info=      Category         Freq            Term        Total  loglift  logprob
term                                                                      
62943  Default  5591.000000              ϲε  5591.000000  30.0000  30.0000
14424  Default  5228.000000     στοχαστικως  5228.000000  29.0000  29.0000
17173  Default  7712.000000       συνεργους  7712.000000  28.0000  28.0000
4093   Default  3326.000000          γενικη  3326.000000  27.0000  27.0000
2010   Default  5986.000000       επισκοπος  5986.000000  26.0000  26.0000
42936  Default  2712.000000        παιονιας  2712.000000  25.0000  25.0000
43333  Default  7707.000000          ροβοαμ  7707.000000  24.0000  24.0000
62631  Default  4062.000000      βραχυτατης  4062.000000  23.0000  23.0000
17175  Default  4257.000000            αλφα  4257.000000  22.0000  22.0000
43314  Default  5227.000000      εγνωριζετο  5227.000000  21.0000  21.0000
17176  Default  4420.000000           δελτα  4420.000000  20.0000  20.0000
17174  Default  4030.000000     εξαιτεισθαι  4030.000000  19.0000  19.0000
14898  Default  2401.000000    ακριβεστερος  2401.000000  18.0000  18.0000
18222  Default  3198.000